In [4]:
import tensorflow as tf
import pyarrow.parquet as pq
import numpy as np


#Después de un par de horas, logré hacer que tensorflow usara mi GPU (GTX 1650 Ti 4Gb VRAM)
if tf.config.list_physical_devices('GPU'):
    print("GPU is available, setting memory growth...")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU found, using CPU instead.")

# Constants
BATCH_SIZE = 2048 #Podría haber intentado 4096 pero llevaba mi memoria al límite y no quería arriesgarme a errores o daños
FEATURES_DIM = 1966

# Similar a en SGD
def load_parquet_batches(parquet_path, batch_size=BATCH_SIZE):
    parquet_file = pq.ParquetFile(parquet_path)

    for batch in parquet_file.iter_batches(batch_size=batch_size):
        features_column = batch['features']
        targets = batch['target'].to_numpy()

        features = np.array([np.array(f.as_py(), dtype=np.float32) for f in features_column], dtype=np.float32)

        yield features, targets

# Propuesta de arquitectura
def create_dnn_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear') 
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    
    return model

model = create_dnn_model(FEATURES_DIM)

# Training Loop
train_file = "train3.parquet"
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    for features_batch, targets_batch in load_parquet_batches(train_file):
        model.fit(features_batch, targets_batch, epochs=1, batch_size=BATCH_SIZE, verbose=1)

# Save Model
model.save("tourism_recommendation_dnn.h5")

print("Training complete and model saved.")


GPU is available, setting memory growth...
Epoch 1/5
1/1 [==============================] - 0s 222ms/step - loss: 1.4230 - mae: 0.9541
Epoch 2/5
1/1 [==============================] - 0s 10ms/step - loss: 1.3802 - mae: 0.9281
Epoch 3/5
1/1 [==============================] - 0s 12ms/step - loss: 1.3367 - mae: 0.9154
Epoch 4/5
1/1 [==============================] - 0s 11ms/step - loss: 1.3062 - mae: 0.8980
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 1.2589 - mae: 0.8811
Training complete and model saved.
